In [3]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
        
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):

        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            
            target = reward
            
            if not done:
                
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
                
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
            
        if self.epsilon > self.epsilon_min:
            
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        
        self.model.load_weights(name)

    def save(self, name):
        
        self.model.save_weights(name)

In [ ]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32
EPISODES = 5000

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

episode: 0/5000, score: 10, e: 1.0
episode: 1/5000, score: 17, e: 1.0
episode: 2/5000, score: 16, e: 0.94
episode: 3/5000, score: 16, e: 0.86
episode: 4/5000, score: 16, e: 0.8
episode: 5/5000, score: 20, e: 0.72
episode: 6/5000, score: 53, e: 0.55
episode: 7/5000, score: 50, e: 0.43
episode: 8/5000, score: 32, e: 0.37
episode: 9/5000, score: 51, e: 0.28
episode: 10/5000, score: 36, e: 0.24
episode: 11/5000, score: 51, e: 0.18
episode: 12/5000, score: 80, e: 0.12
episode: 13/5000, score: 87, e: 0.08
episode: 14/5000, score: 127, e: 0.042
episode: 15/5000, score: 113, e: 0.024
episode: 16/5000, score: 116, e: 0.013
episode: 17/5000, score: 130, e: 0.01
episode: 18/5000, score: 126, e: 0.01
episode: 19/5000, score: 94, e: 0.01
episode: 20/5000, score: 174, e: 0.01
episode: 21/5000, score: 156, e: 0.01
episode: 22/5000, score: 229, e: 0.01
episode: 23/5000, score: 171, e: 0.01
episode: 24/5000, score: 139, e: 0.01
episode: 25/5000, score: 241, e: 0.01
episode: 26/5000, score: 148, e: 0.01

In [12]:
for i_episode in range(100):
    
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    
    for t in range(100):
        
        env.render()
#         print(state)
        action = agent.act(state)
        state, reward, done, info = env.step(action)
        state = np.reshape(state, [1, state_size])
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

Episode finished after 10 timesteps
Episode finished after 10 timesteps
Episode finished after 10 timesteps
Episode finished after 9 timesteps
Episode finished after 10 timesteps
Episode finished after 9 timesteps
Episode finished after 10 timesteps
Episode finished after 9 timesteps
Episode finished after 10 timesteps
Episode finished after 8 timesteps
Episode finished after 9 timesteps
Episode finished after 10 timesteps
Episode finished after 10 timesteps
Episode finished after 9 timesteps
Episode finished after 8 timesteps
Episode finished after 10 timesteps
Episode finished after 10 timesteps
Episode finished after 9 timesteps
Episode finished after 10 timesteps
Episode finished after 9 timesteps
Episode finished after 10 timesteps
Episode finished after 9 timesteps
Episode finished after 9 timesteps
Episode finished after 9 timesteps
Episode finished after 8 timesteps
Episode finished after 9 timesteps
Episode finished after 8 timesteps
Episode finished after 9 timesteps
Episode 